In [29]:
import torch
import numpy as np

In [44]:
'''
A UDF to convert input data into 3-D
array as required for LSTM network.
'''

def temporalize(X, y, lookback):
    output_X = []
    output_y = []
    for i in range(len(X)-lookback-1):
        t = []
        for j in range(1,lookback+1):
            # Gather past records upto the lookback period
            t.append(X[[(i+j+1)], :])
        output_X.append(t)
        output_y.append(y[i+lookback+1])
    return output_X, output_y

In [49]:
timeseries = np.array([
    [0.1**3, 0.2**3, 0.3**3, 0.4**3, 0.5**3, 0.6**3, 0.7**3, 0.8**3, 0.9**3, 1.0**3, 1.1**3, 1.2**3, 1.3**3, 1.4**3, 1.5**3, 1.6**3, 1.7**3, 1.8**3, 1.9**3, 2.0**3],
]).transpose()
timesteps = timeseries.shape[0]
n_features = timeseries.shape[1]
timeseries

array([[1.000e-03],
       [8.000e-03],
       [2.700e-02],
       [6.400e-02],
       [1.250e-01],
       [2.160e-01],
       [3.430e-01],
       [5.120e-01],
       [7.290e-01],
       [1.000e+00],
       [1.331e+00],
       [1.728e+00],
       [2.197e+00],
       [2.744e+00],
       [3.375e+00],
       [4.096e+00],
       [4.913e+00],
       [5.832e+00],
       [6.859e+00],
       [8.000e+00]])

In [91]:
timesteps = 3
X, y = temporalize(X = timeseries, y = np.zeros(len(timeseries)), lookback=timesteps)
X = np.array(X)
X = X.reshape(X.shape[0], n_features, timesteps)
X = torch.Tensor(X)
X.shape

torch.Size([16, 1, 3])

In [113]:
lstm = torch.nn.LSTM(3, 64, 2)
relu = torch.nn.ReLU()

In [114]:
h0 = torch.zeros(2, 1, 64)
c0 = torch.zeros(2, 1, 64)

In [115]:
latent, (hn, cn) = lstm(X, (h0, c0))
latent = relu(latent)
print("output shape: ", latent.shape, "but we need (timesteps)x64 so lets copy the same value x(timesteps):")
#latent = latent.repeat(1,3,1)
#print(latent.shape)

output shape:  torch.Size([16, 1, 64]) but we need (timesteps)x64 so lets copy the same value x(timesteps):


In [116]:
decoder = torch.nn.LSTM(64, 3, 2)
# layer dim, features, hidden dims
h0 = torch.zeros(2, 1, 3)
c0 = torch.zeros(2, 1, 3)

In [118]:
out, (hn, cn) = decoder(latent, (h0, c0))
out = relu(out)
print("hidden layer values: \n", out, " \n and shape: ", out.shape)

hidden layer values: 
 tensor([[[0.0030, 0.0000, 0.0028]],

        [[0.0058, 0.0000, 0.0000]],

        [[0.0126, 0.0000, 0.0000]],

        [[0.0209, 0.0000, 0.0000]],

        [[0.0284, 0.0000, 0.0000]],

        [[0.0343, 0.0000, 0.0000]],

        [[0.0384, 0.0000, 0.0000]],

        [[0.0411, 0.0000, 0.0000]],

        [[0.0430, 0.0000, 0.0000]],

        [[0.0445, 0.0000, 0.0000]],

        [[0.0459, 0.0000, 0.0000]],

        [[0.0476, 0.0000, 0.0000]],

        [[0.0496, 0.0000, 0.0000]],

        [[0.0518, 0.0000, 0.0000]],

        [[0.0542, 0.0000, 0.0000]],

        [[0.0569, 0.0000, 0.0000]]], grad_fn=<ReluBackward0>)  
 and shape:  torch.Size([16, 1, 3])


In [121]:
# Full class:
class AutoEncoder(torch.nn.Module):
    def __init__(self, timesteps: int, hidden_dim: int, layer_dim: int):
        super(AutoEncoder, self).__init__()
        self.hidden_dim = hidden_dim
        self.layer_dim = layer_dim
        self.encoder = torch.nn.LSTM(timesteps, hidden_dim, layer_dim, batch_first=True)
        self.decoder = torch.nn.LSTM(hidden_dim, timesteps, layer_dim)
        self.relu = torch.nn.ReLU()

    def forward(self, x):
        # hidden state
        h0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()
        # cell state
        c0 = torch.zeros(self.layer_dim, x.size(0), self.hidden_dim).requires_grad_()
        latent, (hn, cn) = self.encoder(x, (h0.detach(), c0.detach()))
        latent = self.relu(latent)
        # hidden state
        h0 = torch.zeros(self.layer_dim, x.size(0), self.timesteps).requires_grad_()
        # cell state
        c0 = torch.zeros(self.layer_dim, x.size(0), self.timesteps).requires_grad_()
        out, (hn, cn) = self.decoder(latent, (h0.detach(), c0.detach()))
        return out

In [128]:
# train
X.shape[0]

16